In [5]:
import numpy as np
import pandas as pd
import math
from operator import le

In [6]:
entrada='''Me gustas cuando callas porque estás como ausente,
y me oyes desde lejos, y mi voz no te toca.
Parece que los ojos se te hubieran volado
y parece que un beso te cerrara la boca.

Como todas las cosas están llenas de mi alma
emerges de las cosas, llena del alma mía.
Mariposa de sueño, te pareces a mi alma,
y te pareces a la palabra melancolía.

Me gustas cuando callas y estás como distante.
Y estás como quejándote, mariposa en arrullo.
Y me oyes desde lejos, y mi voz no te alcanza:
déjame que me calle con el silencio tuyo.

Déjame que te hable también con tu silencio
claro como una lámpara, simple como un anillo.
Eres como la noche, callada y constelada.
Tu silencio es de estrella, tan lejano y sencillo.

Me gustas cuando callas porque estás como ausente.
Distante y dolorosa como si hubieras muerto.
Una palabra entonces, una sonrisa bastan.
Y estoy alegre, alegre de que no sea cierto.
'''

In [8]:
print(f'\t-----CODIFICACIÓN LZW-----\n')
# Inicializa diccionario estatico, con las letras ordenadas
diccionario_estatico = dict.fromkeys(sorted(entrada))
# inicializo diccionario estatico con los indices y valores 
diccionario_estatico.update(zip(diccionario_estatico,[format(x,'b') for x in range(len(diccionario_estatico))]))
# Inicializo dataframe dinamico -> Copia diccionario estatico
df = pd.DataFrame([[key, int(diccionario_estatico[key],2), diccionario_estatico[key]] for key in diccionario_estatico.keys()], columns=['letra', 'indice_anterior','indice_binario'])
largo_indice= math.ceil(np.log2(len(df)))  # el largo del bit de cada palabra que se va codificando 
indice_dic_dinamico = len(df)
# Acumulador de letras
acumulador = 0
# indice que recorre el texto, la longitud del texto
indice = 0
# Valor que lleva la codificacion
codificacion = ''
# Ciclo que llena el dataframe con la codificacion de los simbolos
while indice + acumulador < len(entrada):
    # LetraCodigo -> Es  la letra o palabra formada para la codificacion del texto, en caso de entrar en el primer if
    letra_codigo =  entrada[indice:indice+acumulador+1]
    # Letras -> Es la letra que se va comprobando si está en el dinamico o si no está para agregarla
    letra = entrada[indice:indice+acumulador+2]
    if letra not in df.values:
        # se suma 0.5 esto por que por ejemplo si el indice es 64, necesita minimo 7 bit para representarse
        largo_indice = math.ceil(np.log2(indice_dic_dinamico+0.5))
        # Codifica el mensaje añadiendo los bits al inicio y la codificacion del indice de la ultima letra o letras 
        dato_codificado = ('{0:0'+str(largo_indice)+'b}').format(df[df['letra']==letra_codigo].index.values[0])
        # Se agrega el dato al dataframe [letra, valor codificado]
        df.loc[len(df)] = [letra, df[df['letra']==letra_codigo].index.values[0],dato_codificado]
        #Indice -> es con el que accedo a los valores del texto, para obtener las palabras de ahi 
        indice += acumulador + 1
        # indiceDiccionarioDinamico el encargado para calcular la longitud de bits de cada palabra-simbolo-letra codificada
        indice_dic_dinamico += 1
        # acumulador  se reinicia para coger empezar a coger al nueva palabra
        acumulador = 0
        # Se suma la codificacion
        codificacion += dato_codificado
    else:
        # el acomulador se aumenta para ir concatenando en caso de que ya se encuentre la palabra en la lista
        acumulador += 1
# Se agrega el ultimo dato
dato_codificado = ('{0:0'+str(largo_indice)+'b}').format(df[df['letra']==letra_codigo].index.values[0])
df.loc[len(df)] = [letra,df[df['letra']==letra_codigo].index.values[0],dato_codificado]
codificacion += dato_codificado
# Se crea un csv con el dataset
df.to_csv('dataset.csv')


print(f'\n\t\tmensaje codificado:\n {codificacion}')
f = open ('mensaje_codificado.txt','w')
f.write(codificacion)
f.close()
df


	-----CODIFICACIÓN LZW-----


		mensaje codificado:
 00100001000100000101001001111101110101111000110101110100000100111101111100110101100001000001100111000000110101011001011010111000000101101001100101110000110110011111010100000100010101100010001101011110001111001100100101110110110000110101010110010001001100000111100010001000001000000000100001000000100101110101000001100101000011000011000000100100001000011101101100000010010110001000100101010011001001110100000100000001101001100101110010100000000101000000011001010001000000010011000011011010011110000001001111000110010001111000110100000110000000000100100011010011100001000100011110101000100000001010000001011001100010000000010001100101100001001011110001110100101000011100000001001100011111000011100001010000010001000111000011001101101001000110010011101000110101010100100011110001111001011110110111110101000001000000010001111100011000000000010000111001000011011011110010100001111100000111001000111010001110010111100000110110011101011100110111010100000

,letra,indice_anterior,indice_binario
0,\n,0,0
1,,1,1
2,",",2,10
3,.,3,11
4,:,4,100
...,...,...,...
517,a c,167,0010100111
518,cie,345,0101011001
519,ert,141,0010001101
520,to.\n,479,0111011111


In [10]:
from operator import le

print(f'\t-----DECODIFICACIÓN LZW-----\n')

diccionario_estatico = dict.fromkeys(sorted(entrada))
diccionario_estatico.update(zip(diccionario_estatico,[format(x,'b') for x in range(len(diccionario_estatico))]))
largo_indice= math.ceil(np.log2(len(diccionario_estatico)))  # el largo del bit de cada palabra que se va codificando 
indice_dic_dinamico = len(diccionario_estatico)
df = pd.DataFrame([[key, int(diccionario_estatico[key],2), ('{:0{}b}').format(int(diccionario_estatico[key],2), largo_indice)] for key in diccionario_estatico.keys()], columns=['letra', 'indice_anterior','indice_binario'])
indice_letra = '0'
indice = 0
indice2 = largo_indice
letra = ''

	-----DECODIFICACIÓN LZW-----



In [ ]:
'00100001000100000101001001111101110101111000110101110100000100111101111100110101100001000001100111000000110101011001011010111000000101101001100101110000110110011111010100000100010101100010001101011110001111001100100101110110110000110101010110010001001100000111100010001000001000000000100001000000100101110101000001100101000011000011000000100100001000011101101100000010010110001000100101010011001001110100000100000001101001100101110010100000000101000000011001010001000000010011000011011010011110000001001111000110010001111000110100000110000000000100100011010011100001000100011110101000100000001010000001011001100010000000010001100101100001001011110001110100101000011100000001001100011111000011100001010000010001000111000011001101101001000110010011101000110101010100100011110001111001011110110111110101000001000000010001111100011000000000010000111001000011011011110010100001111100000111001000111010001110010111100000110110011101011100110111010100000000000000000000010101001000011011110001100100010000001110110011101001000110011000100011101101000011000111100010001110011100001110010100110100111011010111010110011100000001001110000001011100001101000000000001000101010101000111000001001001011001010111011011001100110000101101010110001101011110010111110001100010100111010110110001011011000000000101101100001001010100001001011010101000001000011110010001010000111101000111011101001000101000000111010100000100100110000110010110010001110000000110101001011001111100001011111010011110111111110000010000110101010001010100111110011111101001010110011010011100111010100101010011101000001110101001010101010100111010000110000100011100010110011011001001110110000000000000100111000101001000101011000101101001000110000110010000110100000110110001110100000111001000111011001010011010110111001000101011001100001001001001011010000010100000101100000110011001001111001110110000001100000000001100001111010110100100010010001111110000010101010111001000110101001001111001100100011000010000011100011011101010110101100011010010011100001111001000011100000011111000111001000011001100011000001010100001010110001011000000101111001011011000011101011001010001100000001100010001100100001100110001101000001101010001101100001101110001110001000101000000111000001110100000011000000100010000001101000000100000000000000010000000100100000010101000001101001010101000000001001111110100110000000110111010111011011001100010011100000010001011010100000011101000010100011010000000001111000010100001101111000011111000100001100101110010101011000000110101001001101001011010011000010000111000101000000010011000001101000001110001011111001110001101001011010001011000100100010011100001000111010011100000011110000011111000000001101010110101011000101011010000000000000001111000111010000011100100001010010101110010011010011010001001011110000100011000010111011101000010001110001100100101010110110000001101101001001000111100010010010011011011000000000011000101010111010000000001110110000000111001111010100011100000110110011110110001101110000001111000010011001010000101010000010010010010100111001010011101101111000101100101010100000001101010110001001110110000001010101110010101110100001001101101011001000110110011001001001000010000101100001111010100001100011100110010001111001011111101001010100111110001010011010000110011111101110001101101011110100000010000101000000101010100010101100000111000110011000110101110011001000111011000111101000111111010011001100011011100010001001001010001001100001001110000010001001110110000000110100010100000101101111001000001100101001011010010010001000111110100100111110001000000110110101010110000000001010001001010001011010001101000111011000010111001000001000011100001110010001110110000001011101111110011101000011111000010100101001001101001110010011111101001000011001100100010011011010100111000011101000011001000011000100100101011011111010011101000101110111001101100101111010110111001010111011010101000010001000010010001111010001100100000111000111111001001111010000101101001011000110001101101011101010100000100010010100111010101100100100011010111011111000000000000'
'001000010001000001010010011111011101011110001101011101000001001111011111001101011000010000011001110000001101010110010110101110000001011010011001011100001101100111110101000001000101011000100011010111100011110011001001011101101100001101010101100100010011000001111000100010000010000000001000010000001001011101010000011001010000110000110000001001000010000111011011000000100101100010001001010100110010011101000001000000011010011001011100101000000001010000000110010100010000000100110000110110100111100000010011110001100100011110001101000001100000000001001000110100111000010001000111101010001000000010100000010110011000100000000100011001011000010010111100011101001010000111000000010011000111110000111000010100000100010001110000110011011010010001100100111010001101010101001000111100011110010111101101111101010000010000000100011111000110000000000100001110010000110110111100101000011111000001110010001110100011100101111000001101100111010111001101110101000000000000000000000101010010000110111100011001000100000011101100111010010001100110001000111011010000110001111000100011100111000011100101001101001110110101110101100111000000010011100000010111000011010000000000010001010101010001110000010010010110010101110110110011001100001011010101100011010111100101111100011000101001110101101100010110110000000001011011000010010101000010010110101010000010000111100100010100001111010001110111010010001010000001110101000001001001100001100101100100011100000001101010010110011111000010111110100111101111111100000100001101010100010101001111100111111010010101100110100111001110101001010100111010000011101010010101010101001110100001100001000111000101100110110010011101100000000000001001110001010010001010110001011010010001100001100100001101000001101100011101000001110010001110110010100110101101110010001010110011000010010010010110100000101000001011000001100110010011110011101100000011000000000011000011110101101001000100100011111100000101010101110010001101010010011110011001000110000100000111000110111010101101011000110100100111000011110010000111000000111110001110010000110011000110000010101000010101100010110000001011110010110110000111010110010100011000000011000100011001000011001100011010000011010100011011000011011100011100010001010000001110000011101000000110000001000100000011010000001000000000000000100000001001000000101010000011010010101010000000010011111101001100000001101110101110110110011000100111000000100010110101000000111010000101000110100000000011110000101000011011110000111110001000011001011100101010110000001101010010011010010110100110000100001110001010000000100110000011010000011100010111110011100011010010110100010110001001000100111000010001110100111000000111100000111110000000011010101101010110001010110100000000000000011110001110100000111001000010100101011100100110100110100010010111100001000110000101110111010000100011100011001001010101101100000011011010010010001111000100100100110110110000000000110001010101110100000000011101100000001110011110101000111000001101100111101100011011100000011110000100110010100001010100000100100100101001110010100111011011110001011001010101000000011010101100010011101100000010101011100101011101000010011011010110010001101100110010010010000100001011000011110101000011000111001100100011110010111111010010101001111100010100110100001100111111011100011011010111101000000100001010000001010101000101011000001110001100110001101011100110010001110110001111010001111110100110011000110111000100010010010100010011000010011100000100010011101100000001101000101000001011011110010000011001010010110100100100010001111101001001111100010000001101101010101100000000010100010010100010110100011010001110110000101110010000010000111000011100100011101100000010111011111100111010000111110000101001010010011010011100100111111010010000110011001000100110110101001110000111010000110010000110001001001010110111110100111010001011101110011011001011110101101110010101110110101010000100010000100100011110100011001000001110001111110010011110100001011010010110001100011011010111010101000001000100101001110101011001001000110101110111110000000000'